In [2]:
import os
from matplotlib import pyplot as plt
%matplotlib

from astropy.time import Time
from astropy.io import ascii as asc

import supernova

Using matplotlib backend: Qt5Agg


In [3]:
GMOS_DIR = '../data/spectra/gmos'
EFOSC_DIR = '../data/spectra/EFOSC'
flist = ['tASASSN-15oz_20160410_Gr13_Free_slit1.5_57723_1_e.fits', 
         'comb20160610_R400.fits', 'comb20160612_B600.fits',
         'tASASSN-15oz_20160802_Gr13_Free_slit1.0_57723_1_e.fits']
dates = ['2016-04-10', '2016-06-10', '2016-06-12', '2016-08-02']
path_flist = [EFOSC_DIR, GMOS_DIR, GMOS_DIR, EFOSC_DIR]
ra = 289.889800 #from SNEX
dec = -33.767000 #from SNEX
redshift = 0.0069

cur_dir = os.getcwd()

lightcurve = supernova.LightCurve2('asassn-15oz')
lightcurve.get_photometry()

In [7]:
fig = plt.figure()
ax1 = fig.add_subplot(2,2,1)
ax2 = fig.add_subplot(2,2,2)
ax3 = fig.add_subplot(2,2,3)
ax4 = fig.add_subplot(2,2,4)
for date_obs, idir, ifile in zip(dates, path_flist, flist):
    spec_phase = (Time(date_obs) - Time(lightcurve.jdexpl, format='jd')).value
    band = ''
    mphot = []
    phot_flux = []
    cenwave_list = []
    for ifilter in lightcurve.jd.keys():
        if len(ifilter) == 1: #avoid swift filters
            cenwave = supernova.get_cenwave(ifilter)
            if (cenwave > 3500):
                mag, date_sep = supernova.get_closest_photometry(Time(date_obs), lightcurve.jd[ifilter], lightcurve.apparent_mag[ifilter])
                if (date_sep <= 7):
                    band=band+ifilter
                    mphot.append(mag)
                    phot_flux.append(supernova.convert_mag_to_flux(mag, ifilter))
                    cenwave_list.append(cenwave)
    filename = '{}_rest_dustcorrsca.dat'.format(ifile.replace('-', '_').split('.')[0])
    tbdata = asc.read(os.path.join(idir, filename), names=['wave', 'flux'])
    if date_obs == '2016-04-10':
        ax = ax1
    elif (date_obs == '2016-06-10') or (date_obs == '2016-06-12'):
        ax = ax2
    else:
        ax = ax3
    ax.plot(tbdata['wave'], tbdata['flux'])
    ax.plot(cenwave_list, phot_flux, 'd')
    ax4.plot(tbdata['wave'], tbdata['flux'])
    ax.set_ylim(min(tbdata['flux'])*0.9, max(tbdata['flux'])*1.1)